In [1]:
import geopandas as gpd
import os
import numpy as np
import pandas as pd

from calitp.tables import tbl
from siuba import *

import utils
import shared_utils

ix = pd.IndexSlice

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
DATA_PATH = "./data/test/"

In [3]:
service = pd.read_parquet(f"{DATA_PATH}shape_frequency_funding.parquet")
processed_shapes = gpd.read_parquet(f'{DATA_PATH}shapes_processed.parquet')

m1 = pd.merge((service 
          >> select(_.calitp_itp_id, _.shape_id, 
                    _.day_name, _.departure_hour, 
                    _.trips_per_hour, _.mean_runtime_min)
         ),
         (processed_shapes
          >> select(_.calitp_itp_id, _.shape_id, _.tract_type)
         ),
         on = ['calitp_itp_id', 'shape_id'],
         how = "outer",
         validate = "m:1",
         indicator=True
)

m1._merge.value_counts()

both          271872
left_only     148536
right_only      4542
Name: _merge, dtype: int64

Is this merge result expected? 

* right only means it has tract_type, but no route info 
* left only means it has route info, but no tract type
* either way, we want to have full info across route info and tract type to be included in this analysis

### Calculate Required Increase

In [4]:
service_combined = pd.read_parquet(f"{DATA_PATH}service_increase.parquet")

In [5]:
hours_by_operator = (service_combined.groupby(['calitp_itp_id', 'tract_type'])
                     [['addl_service_hrs_annual']].sum()
                    )

hours_by_operator.head()

addl_service_hrs_annual
calitp_itp_id tract_type                         
4             rural                  0.000000e+00
              suburban               5.747733e+04
              urban                  2.397386e+06
6             urban                  1.619020e+04
10            rural                  2.219360e+04

### Bus Capital Expenditures

### Methodology notes

* filtered to buses
* all 2019 NTD data
* using 12 year useful life
* bus cost:
    * based on CARB Innovative Clean Transit cost model for a 40', 550kWh electric bus in 2022
    * Inclusive of charger and maintenance infrastructure upgrades, but not potential land aquisition costs for expanded facilities.

In [6]:
hours_by_operator = pd.read_parquet(f"{DATA_PATH}increase_by_operator.parquet")

In [7]:
hours_by_operator.head()

addl_service_hrs_annual  additional_buses  \
calitp_itp_id tract_type                                              
4             rural                  0.000000e+00          0.000000   
              suburban               5.747733e+04         23.477447   
              urban                  2.397386e+06        979.247059   
6             urban                  1.619020e+04          6.613121   
10            rural                  2.219360e+04          9.065296   

                             bus_capex  bus_capex_annualized  
calitp_itp_id tract_type                                      
4             rural       0.000000e+00          0.000000e+00  
              suburban    1.824059e+07          1.302899e+06  
              urban       7.608172e+08          5.434408e+07  
6             urban       5.138005e+06          3.670003e+05  
10            rural       7.043200e+06          5.030857e+05

In [8]:
hours_by_operator.loc[ix[279,:]]

,addl_service_hrs_annual,additional_buses,bus_capex,bus_capex_annualized
tract_type,,,,
urban,501460.266667,204.828686,1.591398e+08,1.136713e+07


### Sandbox

In [9]:
# tbl.transitstacks.provider_info() >> select(_.calitp_itp_id == _.itp_id, _.ntd_id) >> collect() >> inner_join(_, ntd_joined, on='ntd_id')